In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

In [2]:
def reduce_dimen(dataset,column,toreplace):
    for index,i in dataset[column].duplicated(keep=False).iteritems():
        if i==False:
            dataset.set_value(index,column,toreplace)
    return dataset

In [3]:
def act_data_treatment(dataset):
    
    for col in list(dataset.columns):
        if col not in ['people_id', 'activity_id', 'date', 'char_38', 'outcome']:
            if dataset[col].dtype == 'object':
                dataset[col].fillna('type 0', inplace=True)
                dataset[col] = dataset[col].apply(lambda x: x.split(' ')[1]).astype(np.int32)
            elif dataset[col].dtype == 'bool':
                dataset[col] = dataset[col].astype(np.int8)
    
    dataset['year'] = dataset['date'].dt.year
    dataset['month'] = dataset['date'].dt.month
    dataset['day'] = dataset['date'].dt.day
    dataset['isweekend'] = (dataset['date'].dt.weekday >= 5).astype(int)
    dataset = dataset.drop('date', axis = 1)
    
    return dataset

In [4]:
act_train_data = pd.read_csv("act_train.csv",dtype={'people_id': np.str, 'activity_id': np.str, 'outcome': np.int8}, parse_dates=['date'])
act_test_data  = pd.read_csv("act_test.csv", dtype={'people_id': np.str, 'activity_id': np.str}, parse_dates=['date'])
people_data    = pd.read_csv("people.csv", dtype={'people_id': np.str, 'activity_id': np.str, 'char_38': np.int32}, parse_dates=['date'])

In [5]:
act_train_data=act_train_data.drop('char_10',axis=1)
act_test_data=act_test_data.drop('char_10',axis=1)

In [6]:
print('Train data shape:' + format(act_train_data.shape))
print("Test data shape:" + format(act_test_data.shape))
print("People data shape:" + format(people_data.shape))

Train data shape:(2197291, 14)
Test data shape:(498687, 13)
People data shape:(189118, 41)


In [7]:
act_train_data  = act_data_treatment(act_train_data)
act_test_data   = act_data_treatment(act_test_data)
people_data = act_data_treatment(people_data)

In [8]:
train = act_train_data.merge(people_data, on='people_id')
test  = act_test_data.merge(people_data, on='people_id')
del act_train_data
del act_test_data
del people_data

In [9]:
train=train.sort_values(['people_id'])
test=test.sort_values(['people_id'])

In [10]:
train_columns=train.columns.values
test_columns=test.columns.values
features=list(set(train_columns) & set(test_columns))

In [11]:
train.fillna('NA', inplace=True)
test.fillna('NA',inplace=True)
y=train.outcome
train=train.drop('outcome',axis=1)

In [12]:
whole=pd.concat([train,test],ignore_index=True)
categorical=['group_1','activity_category','char_1_x','char_2_x','char_3_x','char_4_x','char_5_x','char_6_x','char_7_x','char_8_x','char_9_x','char_2_y','char_3_y','char_4_y','char_5_y','char_6_y','char_7_y','char_8_y','char_9_y']

In [13]:
for category in categorical:
    whole=reduce_dimen(whole,category,9999999)

In [14]:
X=whole[:len(train)]
X_test=whole[len(train):]
del train
del whole

In [15]:
X=X[features].drop(['people_id','activity_id'], axis=1)
X_test = X_test[features].drop(['people_id', 'activity_id'], axis = 1)

In [16]:
not_categorical=[]
for category in X.columns:
    if category not in categorical:
        not_categorical.append(category)

In [17]:
enc = OneHotEncoder(handle_unknown='ignore')
enc=enc.fit(pd.concat([X[categorical],X_test[categorical]]))
X_cat_sparse=enc.transform(X[categorical])
X_test_cat_sparse=enc.transform(X_test[categorical])

In [18]:
from scipy.sparse import hstack
X_sparse=hstack((X[not_categorical], X_cat_sparse))
X_test_sparse=hstack((X_test[not_categorical], X_test_cat_sparse))

In [19]:
dtrain = xgb.DMatrix(X_sparse,label=y)
dtest = xgb.DMatrix(X_test_sparse)

In [20]:
param = {'max_depth':10, 'eta':0.02, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
param['subsample'] = 0.7
param['colsample_bytree']= 0.7
param['min_child_weight'] = 0
param['booster'] = "gblinear"

watchlist  = [(dtrain,'train')]
num_round = 300
early_stopping_rounds=10
bst = xgb.train(param, dtrain, num_round, watchlist,early_stopping_rounds=early_stopping_rounds)

ypred = bst.predict(dtest)
output = pd.DataFrame({ 'activity_id' : test['activity_id'], 'outcome': ypred })
output.head()
output.to_csv('without_leak.csv', index = False)

[0]	train-auc:0.886945
Will train until train-auc hasn't improved in 10 rounds.
[1]	train-auc:0.895151
[2]	train-auc:0.903531
[3]	train-auc:0.912137
[4]	train-auc:0.920528
[5]	train-auc:0.928519
[6]	train-auc:0.935846
[7]	train-auc:0.942586
[8]	train-auc:0.948834
[9]	train-auc:0.954631
[10]	train-auc:0.959941
[11]	train-auc:0.964739
[12]	train-auc:0.968986
[13]	train-auc:0.972686
[14]	train-auc:0.975862
[15]	train-auc:0.978559
[16]	train-auc:0.980838
[17]	train-auc:0.982751
[18]	train-auc:0.984347
[19]	train-auc:0.985682
[20]	train-auc:0.986803
[21]	train-auc:0.987745
[22]	train-auc:0.988549
[23]	train-auc:0.98924
[24]	train-auc:0.989841
[25]	train-auc:0.99037
[26]	train-auc:0.99084
[27]	train-auc:0.991261
[28]	train-auc:0.99164
[29]	train-auc:0.991984
[30]	train-auc:0.992298
[31]	train-auc:0.992586
[32]	train-auc:0.99285
[33]	train-auc:0.993092
[34]	train-auc:0.993317
[35]	train-auc:0.993524
[36]	train-auc:0.993716
[37]	train-auc:0.993895
[38]	train-auc:0.994062
[39]	train-auc:0.99421